In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[1].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1040,Madden NFL 2005,XB,2004.0,Sports,Electronic Arts,1.61,0.03,0.00,0.08,1.72
1041,Star Ocean: Till The End of Time,PS2,2003.0,Role-Playing,Ubisoft,0.80,0.21,0.55,0.15,1.72
1042,The Sims 3: Late Night Expansion Pack,PC,2010.0,Simulation,Electronic Arts,0.59,0.87,0.00,0.25,1.71
1043,Game Party 3,Wii,2009.0,Puzzle,Warner Bros. Interactive Entertainment,1.43,0.16,0.00,0.12,1.71
1044,Toy Story 2: Buzz Lightyear to the Rescue!,PS,1998.0,Platform,Activision,0.99,0.64,0.00,0.08,1.71


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for _ in range(20):
        driver.execute_script(f"window.scrollBy(0, {value})")
        time.sleep(0.1) #Temps entre chaque scroll

In [5]:
#scrapping part 1
gaming_JVC=gaming_parts[1]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass

    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 

    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'

    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'

    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


100%|██████████| 1038/1038 [9:03:42<00:00, 31.43s/it]  


In [6]:
driver.close()

In [7]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
1040,Madden NFL 2005,XB,2004.0,Sports,Electronic Arts,1.61,0.03,0.00,0.08,1.72,17,(9),18.7,Editeur(s) / Développeur(s) Exient EA Tiburon ...
1041,Star Ocean: Till The End of Time,PS2,2003.0,Role-Playing,Ubisoft,0.80,0.21,0.55,0.15,1.72,17,(105),17.8,Editeur(s) / Développeur(s) Tri-Ace Square Eni...
1042,The Sims 3: Late Night Expansion Pack,PC,2010.0,Simulation,Electronic Arts,0.59,0.87,0.00,0.25,1.71,no_data,no_data,no_data,no_data
1043,Game Party 3,Wii,2009.0,Puzzle,Warner Bros. Interactive Entertainment,1.43,0.16,0.00,0.12,1.71,--,(6),8.5,Editeur(s) / Développeur(s) FarSight Studios W...
1044,Toy Story 2: Buzz Lightyear to the Rescue!,PS,1998.0,Platform,Activision,0.99,0.64,0.00,0.08,1.71,no_data,no_data,no_data,no_data
1045,Deal or No Deal,DS,2007.0,Misc,Mindscape,1.15,0.41,0.00,0.15,1.71,no_data,no_data,no_data,no_data
1046,Ace Combat 5: The Unsung War,PS2,2004.0,Simulation,Sony Computer Entertainment Europe,0.88,0.34,0.32,0.18,1.71,no_data,no_data,no_data,no_data
1047,Need for Speed Underground,XB,2003.0,Racing,Electronic Arts,1.09,0.55,0.00,0.07,1.71,14,(100),17.2,Editeur(s) / Développeur(s) EA Sports BIG Elec...
1048,Kanshuu Nippon Joushikiryoku Kentei Kyoukai: I...,DS,2006.0,Misc,Nintendo,0.00,0.00,1.71,0.00,1.71,no_data,no_data,no_data,no_data


In [ ]:
gaming_JVC.to_csv('gaming_JVC_2.csv')